# CrossTorus example

In this notebook, we run a Navground simulation on the cross torus for illustration purposes.

To start, we load some necessary modules and create some directories as well.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import os

from navground import core, sim
from navground.sim.ui.video import display_video_from_run

import perdiver.perdiver as perdiver
from perdiver.navground_io import parser, run_navground
from perdiver.distances import *

plots_dir = os.path.join("plots", "CrossTorus")
os.makedirs(plots_dir, exist_ok=True)

Next, let us execute the Navground cross torus experiment. We next run the experiment.

In [ ]:
args = parser.parse_args([
        '--scenario', 'CrossTorus',
        '--side', '6.5',
        '--num_steps', '500',
        '--time_step', '0.1',
        '--num_agents', '12',
        '--max_speed', '1.66',
        '--optimal_speed_min', '0.1',
        '--optimal_speed_min', '0.15',
        '--radius', '0.4',
        '--safety_margin', '0.1',
        '--epsilon', '30',
        '--time_delay', '5',
])
behavior_list = ["ORCA"]
runs = {}
for behavior in behavior_list:
    args.behavior = behavior
    runs[behavior] = run_navground(args)

Now, we visualise the ORCA experiment.

In [ ]:
display_video_from_run(run=runs["ORCA"][0], factor=6.0, fps=20)

Now, we plot the two timesteps. For this, we import some necessary modules from the perdiver module. We set the start step as well.

In [ ]:
from perdiver.perdiver import plot_timesteps_cross_torus

args.start_step = 20

fig, ax = plt.subplots(figsize=(5,5))

run = runs["ORCA"][0]
ps = np.array(run.poses)
twists = np.array(run.twists)
X = ps[args.start_step]
Y = ps[args.start_step + args.epsilon]
vel_X = twists[args.start_step]
vel_Y = twists[args.start_step + args.epsilon]
X_len = X.shape[0]-1
# Plot two timesteps
plot_timesteps_cross_torus(run, [args.start_step, args.start_step + args.epsilon], args.side, ax) 
# Save figure
ax.set_xticks([])
ax.set_yticks([])
plt.savefig(os.path.join(plots_dir, "two_timesteps_ORCA.png"))

Now, we plot the corresponding matching diagram for a pair of weights.

In [ ]:
import itertools

from perdiver.distances import  distances_2Dtorus_weighted_velocities
from perdiver.perdiver import get_matching_diagram, plot_matching_diagram, same_diagram_scale

args.weight = 1

fig, ax = plt.subplots(ncols=4, figsize=(16,4))

epsilon_list = [10, 30]
weight_list = [1, 1.8]

for i, (epsilon, weight) in enumerate(itertools.product(epsilon_list, weight_list)):
    args.epsilon = epsilon
    run = runs["ORCA"][0]
    ps = np.array(run.poses)
    twists = np.array(run.twists)
    # Record poses and velocities
    X = ps[args.start_step]
    Y = ps[args.start_step + args.epsilon]
    vel_X = twists[args.start_step]
    vel_Y = twists[args.start_step + args.epsilon]
    X_len = X.shape[0]-1
    # Plot matching diagram
    Dist_X = distances_2Dtorus_weighted_velocities(X, vel_X, weight, args.side)
    Dist_Y = distances_2Dtorus_weighted_velocities(Y, vel_Y, weight, args.side)
    match_diagram = get_matching_diagram(Dist_X, Dist_Y)
    plot_matching_diagram(match_diagram, ax[i], color="blue")
    ax[i].set_title(f"W:{weight}, Eps:{args.epsilon}", fontsize=20)
# end for

same_diagram_scale(ax)
plt.savefig(os.path.join(plots_dir, f"matching_diagrams_two_timesteps.png"))

We can also visualise the evolution of matching diagrams over time.

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(18,4))

args.steps_list = list(range(0, args.num_steps-args.epsilon-args.time_delay, args.time_delay))

### Start by computing the matching diagrams across time
diagrams_parameters = []
run = runs["ORCA"][0]
for i, (epsilon, weight) in enumerate(itertools.product(epsilon_list, weight_list)):
    args.epsilon=epsilon
    ps = np.array(run.poses)
    twists = np.array(run.twists)
    diagrams_list = []
    for idx, start_step in enumerate(args.steps_list):
        if ps.shape[0] > start_step + args.epsilon:
            X = ps[start_step]
            Y = ps[start_step + args.epsilon]
            vel_X = twists[start_step]
            vel_Y = twists[start_step + args.epsilon]
            Dist_X = distances_2Dtorus_weighted_velocities(X, vel_X, weight, args.side)
            Dist_Y = distances_2Dtorus_weighted_velocities(Y, vel_Y, weight, args.side)
            match_diagram = perdiver.get_matching_diagram(Dist_X, Dist_Y)
            diagrams_list.append(match_diagram)
        else: # Add matching diagram from previus loop
            diagrams_list.append(match_diagram)
    # end for over start steps
    diagrams_parameters.append(np.array(diagrams_list))
# end for over behaviors

for i, (epsilon, weight) in enumerate(itertools.product(epsilon_list, weight_list)):
    diagrams_run = diagrams_parameters[i] # Take the run we are considering
    for idx, start_step in enumerate(args.steps_list):
        perdiver.plot_matching_diagram(diagrams_run[idx], ax[i], color=mpl.colormaps["GnBu"](idx/len(args.steps_list)))
    # end for 
    norm = mpl.colors.Normalize(vmin=args.steps_list[0], vmax=args.steps_list[-1])
    cmap = mpl.colormaps["GnBu"]
    mappable = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
    plt.colorbar(mappable=mappable, ax=ax[i])
    ax[i].set_title(f"W:{weight}, Eps:{epsilon}", fontsize=20)
# end for

same_diagram_scale(ax)
plt.savefig(os.path.join(plots_dir, f"matching_diagrams_evolution.png"))

### Cumulative persistence images

We illustrate cumulative persistence images across time. For this, we use the persistence image representation from GUDHI.

In [ ]:
from gudhi import representations

# Load persistence images
npixels = 15
perim = representations.PersistenceImage(resolution=[npixels, npixels], bandwidth=0.1, im_range=[0,3,-3,3])
# Put all 
all_diagrams = []
for diagrams_run in diagrams_parameters:
    all_diagrams += diagrams_run.tolist()
# end for
perim.fit(all_diagrams)
perim_means = []
### Compute persistence images for all behaviors and runs
for diagrams_run in diagrams_parameters: 
    perim_run = perim.transform(diagrams_run)
    perim_means.append(perim_run.mean(axis=0).reshape(15,15))
# end for

## Plot result
fig, ax = plt.subplots(ncols=4, figsize=(16,4), layout="constrained")
for i, (epsilon, weight) in enumerate(itertools.product(epsilon_list, weight_list)):
    ax[i].imshow(perim_means[i])
    ax[i].set_title(f"W:{weight}, Eps:{epsilon}", fontsize=20)
# end for 
# plt.tight_layout()
plt.savefig(os.path.join(plots_dir, f"persistence_images_means.png"))


### Divergence images

We depict divergence images for the four compinations of parameters. We start by computing the persistence images across different runs.

In [ ]:
from perdiver.perdiver import compute_divergence_vector

divergence_runs = []
for i, (epsilon, weight) in enumerate(itertools.product(epsilon_list, weight_list)):
    divergence_list = []
    for matching_diagram in diagrams_parameters[i]:
        divergence_list.append(np.sort(compute_divergence_vector(matching_diagram)))
    # end for
    divergence_runs.append(np.array(divergence_list).transpose())
# end for

Next, we plot the persistence images.

In [ ]:
vmax = np.max(divergence_runs)
vmin = np.min(divergence_runs)

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12,4))
for i, epsilon in enumerate(epsilon_list):
    for j, weigth in enumerate(weight_list):
        divergence_arr = divergence_runs[2*i + j]
        mapable = ax[i,j].imshow(divergence_arr, aspect="auto", vmax=vmax, vmin=vmin, extent=(args.steps_list[0], args.steps_list[-1], 0, X.shape[0]))
        ax[i,j].set_title(f"W:{weight}, Eps:{epsilon}", fontsize=20)
        plt.colorbar(mapable)
        plt.tight_layout()

plt.savefig(os.path.join(plots_dir, f"perdiver_images.png"))

### Absolute divergence

This vectorisation consists in computing the absolute value of persistence divergence.

In [ ]:
from perdiver.perdiver import absolute_perdiver_signal

absdiver_runs = []
for i, diagram in enumerate(diagrams_parameters):
    absdiver_runs.append(
        absolute_perdiver_signal(diagram)
    )
# for over runs

In [ ]:
len(absdiver_runs[0])

In [ ]:
len(args.steps_list)

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))

for i, (epsilon, weight) in enumerate(itertools.product(epsilon_list, weight_list)):
    absdiver = absdiver_runs[i]
    ax.plot(args.steps_list, absdiver, color=mpl.colormaps["Set1"](i/4), label=f"W:{weight}, Eps:{epsilon}")
# end plotting
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(),  ncol=len(behavior_list))
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "absolute-persistence-divergence.png"))